In [1]:
# Three generations seeking romance

In [2]:
# we wil analysis seeking romance  site data make predict age and predict generations models

In [3]:
#libraries

In [4]:
import pandas as pd
import numpy as np
import scipy as sp
import nltk
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [5]:
#read the data

In [13]:
df=pd.read_csv("/content/profiles.csv")

In [11]:
#see first 3 cels

In [14]:
df.head(3)

,age,body_type,diet,drinks,drugs,education,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9,ethnicity,height,income,job,last_online,location,offspring,orientation,pets,religion,sex,sign,smokes,speaks,status
0,22,a little extra,strictly anything,socially,never,working on college/university,about me:<br />\n<br />\ni would love to think...,currently working as an international agent fo...,making people laugh.<br />\nranting about a go...,"the way i look. i am a six foot half asian, ha...","books:<br />\nabsurdistan, the republic, of mi...",food.<br />\nwater.<br />\ncell phone.<br />\n...,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet!<br />\nyou...,"asian, white",75.0,-1.0,transportation,2012-06-28-20-30,"south san francisco, california","doesn&rsquo;t have kids, but might want them",straight,likes dogs and likes cats,agnosticism and very serious about it,m,gemini,sometimes,english,single
1,35,average,mostly other,often,sometimes,working on space camp,i am a chef: this is what that means.<br />\n1...,dedicating everyday to being an unbelievable b...,being silly. having ridiculous amonts of fun w...,NaN,i am die hard christopher moore fan. i don't r...,delicious porkness in all of its glories.<br /...,NaN,NaN,i am very open and will share just about anyth...,NaN,white,70.0,80000.0,hospitality / travel,2012-06-29-21-41,"oakland, california","doesn&rsquo;t have kids, but might want them",straight,likes dogs and likes cats,agnosticism but not too serious about it,m,cancer,no,"english (fluently), spanish (poorly), french (...",single
2,38,thin,anything,socially,NaN,graduated from masters program,"i'm not ashamed of much, but writing public te...","i make nerdy software for musicians, artists, ...",improvising in different contexts. alternating...,my large jaw and large glasses are the physica...,okay this is where the cultural matrix gets so...,movement<br />\nconversation<br />\ncreation<b...,NaN,viewing. listening. dancing. talking. drinking...,"when i was five years old, i was known as ""the...","you are bright, open, intense, silly, ironic, ...",NaN,68.0,-1.0,NaN,2012-06-27-09-10,"san francisco, california",NaN,straight,has cats,NaN,m,pisces but it doesn&rsquo;t matter,no,"english, french, c++",available


In [15]:
#preprocesing

In [16]:
#text cleaning

In [17]:
columns_to_clean = ['education', 'essay0', 'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 'essay7', 'essay8', 'essay9','offspring']
df[columns_to_clean] = df[columns_to_clean].apply(lambda x: x.str.lower().replace('\n', '').replace('\r', '').replace('[^\w\s]', '').replace('\d+', ''))


In [18]:
df.head(3)

,age,body_type,diet,drinks,drugs,education,essay0,essay1,essay2,essay3,essay4,essay5,essay6,essay7,essay8,essay9,ethnicity,height,income,job,last_online,location,offspring,orientation,pets,religion,sex,sign,smokes,speaks,status
0,22,a little extra,strictly anything,socially,never,working on college/university,about me:<br />\n<br />\ni would love to think...,currently working as an international agent fo...,making people laugh.<br />\nranting about a go...,"the way i look. i am a six foot half asian, ha...","books:<br />\nabsurdistan, the republic, of mi...",food.<br />\nwater.<br />\ncell phone.<br />\n...,duality and humorous things,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet!<br />\nyou...,"asian, white",75.0,-1.0,transportation,2012-06-28-20-30,"south san francisco, california","doesn&rsquo;t have kids, but might want them",straight,likes dogs and likes cats,agnosticism and very serious about it,m,gemini,sometimes,english,single
1,35,average,mostly other,often,sometimes,working on space camp,i am a chef: this is what that means.<br />\n1...,dedicating everyday to being an unbelievable b...,being silly. having ridiculous amonts of fun w...,NaN,i am die hard christopher moore fan. i don't r...,delicious porkness in all of its glories.<br /...,NaN,NaN,i am very open and will share just about anyth...,NaN,white,70.0,80000.0,hospitality / travel,2012-06-29-21-41,"oakland, california","doesn&rsquo;t have kids, but might want them",straight,likes dogs and likes cats,agnosticism but not too serious about it,m,cancer,no,"english (fluently), spanish (poorly), french (...",single
2,38,thin,anything,socially,NaN,graduated from masters program,"i'm not ashamed of much, but writing public te...","i make nerdy software for musicians, artists, ...",improvising in different contexts. alternating...,my large jaw and large glasses are the physica...,okay this is where the cultural matrix gets so...,movement<br />\nconversation<br />\ncreation<b...,NaN,viewing. listening. dancing. talking. drinking...,"when i was five years old, i was known as ""the...","you are bright, open, intense, silly, ironic, ...",NaN,68.0,-1.0,NaN,2012-06-27-09-10,"san francisco, california",NaN,straight,has cats,NaN,m,pisces but it doesn&rsquo;t matter,no,"english, french, c++",available


In [19]:
#fill na values

In [20]:
# this script belong to chat gpt
def auto_fill_missing_values(df):
    """
    Automatically fill missing values in a DataFrame.
    """
    # Separate columns by data type
    numerical_columns = df.select_dtypes(include='number').columns
    categorical_columns = df.select_dtypes(exclude='number').columns

    # Fill missing values for numerical columns with the mean
    df[numerical_columns] = df[numerical_columns].fillna(df[numerical_columns].mean())

    # Fill missing values for categorical columns with the mode
    df[categorical_columns] = df[categorical_columns].apply(lambda x: x.fillna(x.mode().iloc[0]))

    return df

# Example usage
df = auto_fill_missing_values(df)


In [21]:
#check

In [22]:
df.isnull().sum()

age            0
body_type      0
diet           0
drinks         0
drugs          0
education      0
essay0         0
essay1         0
essay2         0
essay3         0
essay4         0
essay5         0
essay6         0
essay7         0
essay8         0
essay9         0
ethnicity      0
height         0
income         0
job            0
last_online    0
location       0
offspring      0
orientation    0
pets           0
religion       0
sex            0
sign           0
smokes         0
speaks         0
status         0
dtype: int64

In [23]:
#train test split

In [41]:
x=df[['education', 'offspring','essay0', 'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 'essay7', 'essay8', 'essay9']]
y=df["age"]

In [42]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

In [43]:
# Define stopwords, stemmer, and lemmatizer

In [44]:
text_columns = ['essay0', 'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 'essay7', 'essay8', 'essay9']
categorical_columns = ['education', 'offspring']

In [45]:
# Separate text and categorical features
x_train_text = x_train[text_columns].astype(str)
x_train_categorical = x_train[categorical_columns].astype(str)

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a TF-IDF vectorizer for text data
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')

In [47]:
# Fit and transform the training text data
x_train_text_tfidf = tfidf_vectorizer.fit_transform(x_train_text['essay0'])  # Example for 'essay0'

In [48]:
from sklearn.preprocessing import OneHotEncoder

# Create a one-hot encoder for categorical data
onehot_encoder = OneHotEncoder()

In [49]:
# Fit and transform the training categorical data
x_train_categorical_encoded = onehot_encoder.fit_transform(x_train_categorical)

In [50]:
from scipy.sparse import hstack

# Combine text and categorical features
x_train_processed = hstack([x_train_text_tfidf, x_train_categorical_encoded])

In [51]:
from sklearn.linear_model import LinearRegression

# Initialize and train the Linear Regression model
reg_model = LinearRegression()
reg_model.fit(x_train_processed, y_train)

LinearRegression()

In [55]:
#prepare test data
x_test_text = x_test[text_columns].astype(str)
x_test_categorical = x_test[categorical_columns].astype(str)
x_test_text_tfidf = tfidf_vectorizer.transform(x_test_text['essay0'])
x_test_categorical_encoded = onehot_encoder.transform(x_test_categorical)
x_test_processed = hstack([x_test_text_tfidf, x_test_categorical_encoded])

In [56]:
from sklearn.metrics import r2_score

# Make predictions on the testing set
predictions = reg_model.predict(x_test_processed)

# Evaluate the model using R^2
r2 = r2_score(y_test, predictions)
print("R^2 Score:", r2)

R^2 Score: -0.433809101395227


In [57]:
#model success

In [58]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)

print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)


MAE: 7.797627734265983
MSE: 110.35542482588671
RMSE: 10.50501903024867


In [59]:
#Clasification

In [60]:
#  prepare x , y

In [73]:
# Classify generation based on age
df['Generation'] = df['age'].apply(lambda age: 'Millennial' if 18 <= age <= 38 else 'Gen X-er' if 39 <= age <= 54 else 'Boomer')
y=df['Generation']
x=df[['education', 'offspring','essay0', 'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 'essay7', 'essay8', 'essay9']]

In [74]:
#train test split

In [75]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.20,random_state=42)

In [78]:
#same on

In [76]:
text_columns = ['essay0', 'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 'essay7', 'essay8', 'essay9']
categorical_columns = ['education', 'offspring']
# Separate text and categorical features
x_train_text = x_train[text_columns].astype(str)
x_train_categorical = x_train[categorical_columns].astype(str)
from sklearn.feature_extraction.text import TfidfVectorizer
# Create a TF-IDF vectorizer for text data
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
# Fit and transform the training text data
x_train_text_tfidf = tfidf_vectorizer.fit_transform(x_train_text['essay0'])
from sklearn.preprocessing import OneHotEncoder
# Create a one-hot encoder for categorical data
onehot_encoder = OneHotEncoder()
# Fit and transform the training categorical data
x_train_categorical_encoded = onehot_encoder.fit_transform(x_train_categorical)
from scipy.sparse import hstack
# Combine text and categorical features
x_train_processed = hstack([x_train_text_tfidf, x_train_categorical_encoded])




#prepare test data
x_test_text = x_test[text_columns].astype(str)
x_test_categorical = x_test[categorical_columns].astype(str)
x_test_text_tfidf = tfidf_vectorizer.transform(x_test_text['essay0'])
x_test_categorical_encoded = onehot_encoder.transform(x_test_categorical)
x_test_processed = hstack([x_test_text_tfidf, x_test_categorical_encoded])

In [79]:
#model

In [80]:
nb=MultinomialNB()

In [81]:
#fiting

In [82]:
model=nb.fit(x_train_processed,y_train)

In [ ]:
# make predictions

In [84]:
pred=nb.predict(x_test_processed)

In [ ]:
#model summary

In [87]:
from sklearn.metrics import accuracy_score
accuracy_score(pred,y_test)

0.7980295566502463

In [88]:
#save model

In [89]:
import joblib

In [90]:
joblib.dump(model, 'model.joblib')

['model.joblib']

In [91]:
#sources

In [92]:
#Zafer Acar notes
#[https://chat.openai.com/]